<center>
    <tr>
    <td><img src="images/Quansight_Logo_Lockup_1.png" width="25%"></img></td>
    </tr>
</center>

---

# Neural Networks

## Outline

- Definition of neural networks
- Essential components: activation functions, layers, forward & backward propagation
- Building a neural network in NumPy
  - Hand-coding gradient descent

## Definition of Neural Network

*Neural network*: a *function* defined by following components:

+ Resulting function: compute $\hat{y}=f(x)$ by $L$ successive function compositions:

  $$\begin{aligned}
  a^{0} &= x &&\text{(input feature vector)}\\
  z^{\ell} &= W^{\ell}a^{\ell-1}+b^{\ell} \\
  a^{\ell}  &= g_{\ell}\left( z^{\ell} \right) &&(\ell=1,\dotsc,L) \\
  \hat{y} &=a^{L} && \text{(vector from output layer)}
  \end{aligned}$$

+ Notation here from Michael Neilsen's [*Neural Networks and Deep Learning*](http://neuralnetworksanddeeplearning.com)
+ $W^\ell$ and $b^\ell$ are *weight matrices* & *bias vectors* respectively associated with layer $\ell$ of the network
+ Entry $w^{\ell}_{jk}$ of matrix $W^\ell$ is the weight parameter associated with the link connecting the $k$th neuron in layer $\ell-1$ to the $j$th neuron in layer $\ell$

<center>
    <a href="http://neuralnetworksanddeeplearning.com/chap2.html"><img src="images/tikz16.png" width="750px"></img></a>
</center>

+ Vectors $z^{\ell}\in\mathbb{R}^{n_{\ell}\times 1}$ are *weighted inputs*
+ Vectors $a^{\ell}\in\mathbb{R}^{n_{\ell}\times 1}$ are *activations*
+ Functions $g_{\ell}$ are *activation functions*

#### Example Neural Network
<center>
    <img src="./images/NN-00.png" width="750px"></img>
</center>

+ Example has 2 hidden layers & an output layer, so $L=3$ layers
+ Number of units in each layer are $$n_{0}=5, n_{1}=3, n_{2}=4, n_{3}=2$$
+ Each layer has an activation functions: $g_{1}$, $g_{2}$, $g_{3}$
  + Hidden layer 1 has weight matrix $W^{1}\in\mathbb{R}^{3\times 5}$, bias vector $b^{1}\in\mathbb{R}^{3\times1}$
  + Hidden layer 2 has weight matrix $W^{2}\in\mathbb{R}^{4\times 3}$, bias vector $b^{2}\in\mathbb{R}^{4\times1}$
  + Output layer has weight matrix $W^{3}\in\mathbb{R}^{2\times 4}$, bias vector $b^{3}\in\mathbb{R}^{2\times1}$
   $$\Rightarrow (5+1)\times3 + (3+1)\times4 + (4+1)\times2 = 44\ \text{parameters}$$

## How to set up/use Neural Network

1. Choose *architecture* of neural network:
   + $L$ (# hidden layers + output layer)
   + $n_{0}, \dotsc, n_{L}$ (# units input + in layers)
   + $g_{1}, \dotsc, g_{L}$ (activation functions for each layer)
2. Choose appropriate *objective* or *loss function*
3. Solve corresponding optimization problem ("fitting to training data")

+ Hidden layers & nonlinear activation functions encode *feature engineering*
+ A lot of experimentation typically required to get appropriate architecture

## Activation functions
<center>
    <img src="./images/signum.png"></img>
</center>

+ $\text{sign}$ (sometimes called "signum" or "step" function) defined as
$$ \displaystyle{\text{sign}(t) := \begin{cases} +1, & \text{if $t\ge0$}\\ -1, & \text{if $t<0$} \end{cases}} $$
+ used in *perceptron* (earliest classifier)

<center>
    <img src="./images/sigmoid.png"></img>
</center>

+ $\sigma$ ("sigmoid" or "logistic" function) defined as
   $$ \displaystyle{\sigma(t) := \frac{e^{t}}{1 + e^{t}}} $$
+ commonly used for output layer activation function *logistic regression* (e.g, *binary classification* models)

<center>
    <img src="./images/id.png"></img>
</center>

+ $\text{id}$ (identity function) defined as
$$ \displaystyle{\text{id}(t) := t} $$
+ implicitly used as output layer activation function in *regression* models

+ $\text{softmax}$: maps vectors to vectors of same length 
+ Given $t\in\mathbb{R}^{K}$, $\text{softmax}(t)$ defined by
  $$ \displaystyle{[\text{softmax}(t)]_{k} := \frac{e^{t_{k}}}{\sum_{i=1}^{K} e^{t_{i}}}} $$
+ $\text{softmax}(t)$ has nonnegative values summing to one (probabilities)
+ Used for *multi-class classification* problems

## Loss functions

+ For *regression* problems, typically *mean-squared error* as loss:
  $$ \mathcal{L}(\hat{y}, y) = \frac{1}{N} \sum_{k=1}^{N}  \left[ y_{k} - \hat{y}_{k} \right]^2 $$

+ For *classification* problems, typically *cross-entropy* (or *log loss*) as loss:
  $$ \mathcal{L}(\hat{y}, y) = -\sum_{k=1}^{N} \left[ y_{k} \log\left( \hat{y}_{k} \right) + \left(1-y_{k}\right) \log\left(1- \hat{y}_{k} \right) \right]$$
  (extension exists for multi-class classification problems)

#### Remarks

+ *Convolutional* neural networks use *convolution matrices* (e.g., image/signal processing)
+ *Recurrent* neural networks permit *loops* (e.g., text processing)
+ *Backpropagation*: strategy for computing gradients for iterative optimizers
+ *Deep learning*: loosely, neural network with several hidden layers
+ Vast number of frameworks (e.g., TensorFlow, Theano, PyTorch, etc.); start with those first

---

## Building a neural network in NumPy

+ use same activation function in all layers: the *logistic* or *sigmoid* function
+ use a functional programming style to help build intuition
  + introduce dictionary `model` to store all data associated with the neural network (weight matrices, bias vectors, etc.)
+ production codes typically use object-oriented style
+ production codes optimized for efficiency (unlike what we'll develop here)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib

### Exercise: Implement activation function(s) & their derivatives

For today's purposes, we'll use the same activation function in every layer of the network, namely the *logistic* or *sigmoid* function:
$$ \sigma(x) = \frac{1}{1+\exp(-x)} = \frac{\exp(x)}{1+\exp(x)}.$$
A bit of calculus shows that
$$ \sigma'(x) = \sigma(x)(1-\sigma(x)) .$$

Actually, a more numerically robust formula for $\sigma(x)$ (i.e., one that works for large positive or large negative input equally well) is
$$
\sigma(x) = \begin{cases} \frac{1}{1+\exp(-x)} & (x\ge0) \\ 1 - \frac{1}{1+\exp(x)} & \mathrm{otherwise} \end{cases}.
$$

In [ ]:
def sigma(x):
    '''The logistic function; accepts arbitrary arrays as input (vectorized)'''
    # your code here...
def sigma_prime(x):
    '''The *derivative* of the logistic function; accepts arbitrary arrays as input (vectorized)'''
    # your code here...

**Solution:**
```python
def sigma(x):
    '''The logistic function; accepts arbitrary arrays as input (vectorized)'''
    return np.where(x>=0, 1/(1+np.exp(-x)), 1 - 1/(1+np.exp(x))) # piecewise-defined for numerical robustness
def sigma_prime(x):
    '''The *derivative* of the logistic function; accepts arbitrary arrays as input (vectorized)'''
    return sigma(x)*(1-sigma(x)) # Derivative of logistic function
```

### Exercise: Implement loss function & its gradient

For the loss function, we'll use the typical "$L_2$-norm of the error" (alternatively called *mean-square error (MSE)* when averaged over a batch of values):
$$ \mathcal{L}(\hat{y},y) = \frac{1}{2} \|\hat{y}-y\|^{2} = \frac{1}{2} \sum_{k=1}^{d} \left[ \hat{y}_{k}-y_{k} \right]^{2}.$$
Again, using multivariable calculus, we can see that
$$\nabla_{\hat{y}} \mathcal{L}(\hat{y},y) = \hat{y} - y.$$

Implement both of these functions below.

In [ ]:
def loss(yhat, y):
    '''The loss as measured by the L2-norm squared of the error'''
    # your code here...
def loss_prime(yhat, y):
    '''Implementation of the gradient of the loss function'''
    # your code here...

**Solution:**
```python
def loss(yhat, y):
    '''The loss as measured by the L2-norm squared of the error'''
    return 0.5 * np.square(yhat-y).sum()
def loss_prime(yhat, y):
    '''Implementation of the gradient of the loss function'''
    return (yhat - y) # gradient w.r.t. yhat
```

### Exercise: Create an initialization function to set up model as a dict

Write a function `initialize_model` that accepts a list  `dimensions` of positive integer inputs that constructs a `dict` with specific key-value pairs:
+ `model['nlayers']` : number of layers in neural network
+ `model['weights']` : list of NumPy matrices with appropriate dimensions
+ `model['biases']` : list of NumPy (column) vectors of appropriate dimensions
+ `model['act_funs']` : list of (vectorised) Python functions (all `sigma` for now)
+ `model['act_fun_grads']`: list of (vectorised) Python functions (all `sigma_prime` for now)
+ The matrices in `model['weights']` and the vectors in `model['biases']` should be initialized as randomly arrays of the appropriate shapes.

If the input list `dimensions` has `L+1` entries, the number of layers is `L` (the first entry of `dimensions` is the input dimension, the next ones are the number of units/neurons in each subsequent layer going up to the output layer).
Thus, for example:

```python
>>> dimensions = [784, 15, 10]
>>> model = initialize_model(dimensions)
>>> for k, (W, b) in enumerate(zip(model['weights'], model['biases'])):
>>>    print(f'Layer {k+1}:\tShape of W{k+1}: {W.shape}\tShape of b{k+1}: {b.shape}')
```
```
Layer 1:	Shape of W1: (15, 784)	Shape of b1: (15, 1)
Layer 2:	Shape of W2: (10, 15)	Shape of b2: (10, 1)
```

In [ ]:
def initialize_model(dimensions, act_funs=[sigma], act_fun_grads=[sigma_prime]):
    '''Accepts a list of positive integers; returns a dict 'model' with key/values as follows:
      model['nlayers']  : number of layers in neural network
      model['weights']  : list of NumPy matrices with appropriate dimensions
      model['biases']   : list of NumPy (column) vectors of appropriate dimensions
    These correspond to the weight matrices & bias vectors associated with each layer of a neural network.'''
    # your code here...

**Solution:**
```python
def initialize_model(dimensions, act_funs=[sigma], act_fun_grads=[sigma_prime]):
    '''Accepts a list of positive integers; returns a dict 'model' with key/values as follows:
      model['nlayers']  : number of layers in neural network
      model['weights']  : list of NumPy matrices with appropriate dimensions
      model['biases']   : list of NumPy (column) vectors of appropriate dimensions
    These correspond to the weight matrices & bias vectors associated with each layer of a neural network.'''
    weights, biases = [], []
    L = len(dimensions) - 1 # number of layers (i.e., excludes input layer)
    for l in range(L):
        W = np.random.randn(dimensions[l+1], dimensions[l])
        b = np.random.randn(dimensions[l+1], 1)
        weights.append(W)
        biases.append(b)
    act_funs, grads = [sigma for ell in range(L)], [sigma_prime for ell in range(L)]
    return dict(weights=weights, biases=biases, nlayers=L, act_funs=act_funs, act_fun_grads=grads)
```

In [ ]:
# Use a test example to illustrate that the network is initialized as needed
dimensions = [784, 15, 10]
model = initialize_model(dimensions)
for k, (W, b) in enumerate(zip(model['weights'], model['biases'])):
    print(f'Layer {k+1}:\tShape of W{k+1}: {W.shape}\tShape of b{k+1}: {b.shape}')

In [ ]:
# Let's examine the weight matrix & bias vector associated with the second layer.
print(f'W2:\n\n{model["weights"][1]}')  # Expect a 10x15 matrix of random numbers
print(f'b2:\n\n{model["biases"][1]}')   # Expect a 10x1 vector of random numbers

In [ ]:
print(model['act_funs'])         # Function objects all point to same reference
print(model['act_fun_grads'])

### Exercise: Implement a function for forward propagation

Write a function `forward` that uses the architecture described in a `dict` as created by `initialize_model` to evaluate the output of the neural network for a given input *column* vector `x`.
+ Take $a^{0}=x$ from the input. **Assume the input has features along rows** (the opposite of what we've seen so far).
+ For $\ell=1,\dotsc,L$, compute & store the intermediate computed vectors $z^{\ell}=W^{\ell}a^{\ell-1}+b^{\ell}$ (the *weighted inputs*) and $a^{\ell}=g^{\ell}\left(z^{\ell}\right)$ (the *activations*) in an updated dictionary `model`. That is, modify the input dictionary `model` so as to accumulate:
  + `model['activations']`: a list with entries $a^{\ell}$ for $\ell=0,\dotsc,L$
  + `model['weighted_inputs']`: a list with entries $z^{\ell}$ for $\ell=1,\dotsc,L$
+ The function should return the computed output $a^{L}$ and the modified dictionary `model`.
Notice that input `x` can be a matrix of dimension $n_{0} \times N_{\mathrm{batch}}$ corresponding to a batch of input vectors (here, $n_0$ is the dimension of the expected input vectors).

In [ ]:
# Abstract process into function and run tests again.
def forward(x, model):
    '''Implementation of forward propagation through a feed-forward neural network.
       x : input array oriented column-wise (i.e., features along the rows)
       model : dict with same keys as output of initialize_model & appropriate lists in 'weights' & 'biases'
    The output dict model is the same as the input with additional keys 'z_inputs' & 'activations';
    these are accumulated to be used later for backpropagation. Notice the lists model['z_inputs'] &
    model['activations'] both have the same number of entries as model['weights'] & model['biases']
    (one for each layer).
    '''
    # your solution here...

**Solution:**
```python
def forward(x, model):
    '''Implementation of forward propagation through a feed-forward neural network.
       x : input array oriented column-wise (i.e., features along the rows)
       model : dict with same keys as output of initialize_model & appropriate lists in 'weights' & 'biases'
    The output dict model is the same as the input with additional keys 'z_inputs' & 'activations';
    these are accumulated to be used later for backpropagation. Notice the lists model['z_inputs'] &
    model['activations'] both have the same number of entries as model['weights'] & model['biases']
    (one for each layer).
    '''
    a = x
    activations = [a]
    weighted_inputs = []
    for W, b, g in zip(model['weights'], model['biases'], model['act_funs']):
        z = W @ a + b
        a = g(z)
        weighted_inputs.append(z)
        activations.append(a)
    model['activations'], model['weighted_inputs'] = activations, weighted_inputs
    return (a, model)
```

In [ ]:
# Use a test example to illustrate that the network is initialized as needed
dimensions = [784, 15, 10]
model = initialize_model(dimensions)
print(f'Before executing *forward*:\nkeys == {model.keys()}')

In [ ]:
N_batch = 3  # Let's use, say, 3 random inputs & their corresponding outputs
x_input = np.random.rand(dimensions[0], N_batch)
y = np.random.rand(dimensions[-1], N_batch)

In [ ]:
y_hat, model = forward(x_input, model)  # the dict model is *updated* by forward propagation
print(f'After executing *forward*:\nkeys == {model.keys()}')
# Observe additional dict keys: 'activations' & 'z_inputs'

### Algorithm for backpropagation:

#### (optional reading for the mathematically brave)

The description here is based on the *wonderfully concise* description from Michael Neilsen's [*Neural Networks and Deep Learning*](http://neuralnetworksanddeeplearning.com/chap2.html). Neilsen has artfully crafted a summary using the bare minimum mathematical prerequisites. The notation elegantly summarises the important ideas in a way to make implementation easy in array-based frameworks like Matlab or NumPy. This is the best description I know of that does this.

In the following, $\mathcal{L}$ is the loss function and the symbol $\odot$ is the [*Hadamard product*](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)) of two conforming arrays; this is simply a fancy way of writing the usual element-wise product of arrays as computed by NumPy & is sometimes called the *Schur product*. This can be reformulated in usual matrix algebra for analysis.

Given a neural network with $L$ layers (not including the "input layer") described by an appropriate architecture:

1. Input $x$: Set the corresponding activation $a^{0} \leftarrow x$ for the input layer.
2. Feedforward: For each $\ell=1,2,\dotsc,L$, compute *weighted inputs* $z^{\ell}$ & *activations* $a^{\ell}$ using the formulas
$$
\begin{aligned}
z^{\ell} & \leftarrow  W^{\ell} a^{\ell-1} + b^{\ell}, \\
a^{\ell} & \leftarrow  g_{\ell} \left( z^{\ell}\right)
\end{aligned}.
$$
3. Starting from the end, compute the "error" in the output layer $\delta^{L}$ according to the formula
$$
\delta^{L} \leftarrow \nabla_{a^{L}} \mathcal{L} \odot g_{L}'\left(z^{L}\right)
$$

4. *Backpropagate* the "error" for $\ell=L−1\dotsc,1$ using the formula
$$
\delta^{\ell} \leftarrow \left[ W^{\ell+1}\right]^{T}\delta^{\ell+1} \odot g_{\ell}'\left(z^{\ell}\right).
$$
5. The required gradients of the loss function $\mathcal{L}$ with respect to the parameters $W^{\ell}_{p,q}$ and $b^{\ell}_{r}$ can be computed directly from the "errors" $\left\{ \delta^{\ell} \right\}$ and the weighted inputs $\left\{ z^{\ell} \right\}$ according to the relations
$$
\begin{aligned}
   \frac{\partial \mathcal{L}}{\partial W^{\ell}_{p,q}} &= a^{\ell-1}_{q} \delta^{\ell}_{p} &&(\ell=1,\dotsc,L)\\
   \frac{\partial \mathcal{L}}{\partial b^{\ell}_{r}} &= \delta^{\ell}_{r} &&
\end{aligned}
$$

### Exercise: Implement a function for backward propagation

Implement a function `backward` that implements the back-propagation algorithm to compute the gradients of the loss function $\mathcal{E}$ with respect to the weight matrices $W^{\ell}$ and the bias vectors $b^{\ell}$.
+ The function should accept a column vector `y` of output labels and an appropriate dictionary `model` as input.
+ The dict `model` is assumed to have been generated *after* a call to `forward`; that is, `model` should have keys `'w_inputs'` and `'activations'` as computed by a call to `forward`.
+ The result will be a modified dictionary `model` with two additional key-value pairs:
  + `model['grad_weights']`: a list with entries $\nabla_{W^{\ell}} \mathcal{L}$ for $\ell=1,\dotsc,L$
  + `model['grad_biases']`: a list with entries $\nabla_{b^{\ell}} \mathcal{L}$ for $\ell=1,\dotsc,L$
+ Notice the dimensions of the matrices $\nabla_{W^{\ell}} \mathcal{L}$ and the vectors $\nabla_{b^{\ell}} \mathcal{L}$ will be identical to those of ${W^{\ell}}$ and ${b^{\ell}}$ respectively.
+ The function's return value is the modified dictionary `model`.


Notice that input `y` can be a matrix of dimension $n_{L} \times N_{\mathrm{batch}}$ corresponding to a batch of output vectors (here, $n_L$ is the number of units in the output layer).

In [ ]:
def backward(y, model):
    '''Implementation of backward propagation of data through the network
       y : output array oriented column-wise (i.e., features along the rows) as output by forward
       model : dict with same keys as output by forward
    Note the input needs to have keys 'nlayers', 'weights', 'biases', 'z_inputs', and 'activations'
    '''
    # your code here...

**Solution:**
```python
def backward(y, model):
    '''Implementation of backward propagation of data through the network
       y : output array oriented column-wise (i.e., features along the rows) as output by forward
       model : dict with same keys as output by forward
    Note the input needs to have keys 'nlayers', 'weights', 'biases', 'z_inputs', and 'activations'
    '''
    Nbatch = y.shape[1] # Needed to extend for batches of vectors
    # Compute the "error" delta^L for the output layer
    yhat = model['activations'][-1]
    z, a = model['weighted_inputs'][-1], model['activations'][-2]
    delta = loss_prime(yhat, y) * model['act_fun_grads'][-1](z)
    # Use delta^L to compute gradients w.r.t b & W in the output layer.
    grad_b, grad_W = delta @ np.ones((Nbatch, 1)), np.dot(delta, a.T)
    grad_weights, grad_biases = [grad_W], [grad_b]
    loop_iterates = zip(model['weights'][-1:0:-1],
                        model['weighted_inputs'][-2::-1],
                        model['activations'][-3::-1],
                        model['act_fun_grads'][-2::-1])
    for W, z, a, g_prime in loop_iterates:
        delta = np.dot(W.T, delta) * g_prime(z)
        grad_b, grad_W = delta @ np.ones((Nbatch, 1)), np.dot(delta, a.T)
        grad_weights.append(grad_W)
        grad_biases.append(grad_b)
    # We built up lists of gradients backwards, so we reverse the lists
    model['grad_weights'], model['grad_biases'] = grad_weights[::-1], grad_biases[::-1]
    return model
```

In [ ]:
# Use the test example from above. Assume model, x_input have been initialized & *forward* has been executed already.
print(f'Before executing *backward*:\nkeys == {model.keys()}')

In [ ]:
model = backward(y, model)  # the dict model is updated *again* by backward propagation
print(f'After executing *backward*:\nkeys == {model.keys()}')
# Observe additional dict keys: 'grad_weights' & 'grad_biases'

### Exercise: Implement a function to update the model parameters using computed gradients.

Given some positive learning rate $\eta>0$, we want to change all the weights and biases using their gradients.
Write a function `update` to compute a single step of gradient descent assuming that the model gradients have been computed for a given input vector.
+ The functions signature should be `update(eta, model)` where `eta` is a positive scalar value and `model` is a dictionary as output from `backward`.
+ The result will be an updated model with the values updated for `model['weights']` and `model['biases']`.
+ Written using array notations, these updates can be expressed as
   $$
   \begin{aligned}
   W^{\ell} &\leftarrow W^{\ell} - \eta \nabla_{W^{\ell}} \mathcal{L} &&(\ell=1,\dotsc,L)\\
   b^{\ell} &\leftarrow b^{\ell} - \eta \nabla_{b^{\ell}} \mathcal{L} &&
   \end{aligned}.
   $$
+ Written out component-wise, the preceding array expressions would be written as
   $$
   \begin{aligned}
      W^{\ell}_{p,q} &\leftarrow W^{\ell}_{p,q} - \eta \frac{\partial \mathcal{L}}{\partial W^{\ell}_{p,q}}
      &&(\ell=1,\dotsc,L)\\
      b^{\ell}_{r} &\leftarrow b^{\ell}_{r} - \eta \frac{\partial \mathcal{L}}{\partial b^{\ell}_{r}} &&
   \end{aligned}
   $$.
+ For safety, have the update step delete the keys added by calls to `forward` and `backward`, i.e., the keys `'z_inputs'`, `'activations'`, `'grad_weights'`, & `'grad_biases'`.
+ The output should be a dict `model` like before.

In [ ]:
def update(eta, model):
    '''Use learning rate and gradients to update model parameters
       eta : learning rate (positive scalar parameter)
       model : dict with same keys as output by backward
    Output result is a modified dict model
    '''
    # your code here...

**Solution:**
```python
def update(eta, model):
    '''Use learning rate and gradients to update model parameters
       eta : learning rate (positive scalar parameter)
       model : dict with same keys as output by backward
    Output result is a modified dict model
    '''
    new_weights, new_biases = [], []
    for W, b, dW, db in zip(model['weights'], model['biases'], model['grad_weights'], model['grad_biases']):
        new_weights.append(W - (eta * dW))
        new_biases.append(b- (eta * db))
    model['weights'] = new_weights
    model['biases'] = new_biases
    # Get rid of extraneous keys/values
    for key in ['weighted_inputs', 'activations', 'grad_weights', 'grad_biases']:
        del model[key]
    return model
```

In [ ]:
# Use the test example from above. Assume *forward* & *backward* have been executed already.
print(f'Before executing *update*:\nkeys == {model.keys()}')

In [ ]:
eta = 0.5  # Choice of learning rate
model = update(eta, model)  # the dict model is updated *again* by calling *update*
print(f'After executing *update*:\nkeys == {model.keys()}')
# Observe fewer dict keys: extraneous keys have been freed.

In [ ]:
# Observe the required sequence of executions: (forward -> backward -> update -> forward -> backward -> ...)
# If done out of sequence, results in KeyError
backward(y, model)  # This should cause an exception (KeyError)

### Exercise: Implement steepest descent in a loop for random training data

Let's now attempt to use our NumPy-based model to implement the steepest descent algorithm. We'll explain these numbers shortly in the context of the MNIST digit classification problem.

+ Generate random arrays `X` and `y` of dimensions $28^2 \times N_{\mathrm{batch}}$ and $10\times N_{\mathrm{batch}}$ respectively where $N_{\mathrm{batch}}=10$.
+ Initialize the network architecture using `initialize_model` as above to require an input layer of $28^2$ units, a hidden layer of 15 units, and an output layer of 10 units.
+ Choose a learning rate of, say, $\eta=0.1$ and a number of epochs `n_epoch` of, say, $30$.
+ Construct a for loop with `n_epochs` iterations in which:
    + The output `yhat` is computed from the input`X` using `forward`.
    + The function `backward` is called to compute the gradients of the loss function with respect to the weights and biases.
    + Update the network parameters using the function `update`.
    + Compute and print out the epoch (iteration counter) and the value of the loss function.

In [ ]:
N_batch = 10
n_epochs = 30
dimensions = [784, 15, 10]
X = np.random.rand(dimensions[0], N_batch)
y = np.random.rand(dimensions[-1], N_batch)
eta = 0.1
model = ...

for epoch in range(n_epochs):
    pass
    # your code here ...
 # Expect to see loss values decreasing systematically in each iteration.

**Solution:**
```python
N_batch = 10
n_epochs = 30
dimensions = [784, 15, 10]
X = np.random.rand(dimensions[0], N_batch)
y = np.random.rand(dimensions[-1], N_batch)
eta = 0.1
model = initialize_model(dimensions)

for epoch in range(n_epochs):
    yhat, model = forward(X, model)
    err = loss(yhat, y)
    print(f'Epoch: {epoch}\tLoss: {err}')
    model = backward(y, model)
    model = update(eta, model)

 # Expect to see loss values decreasing systematically in each iteration.
```

### Exercise: Modify the steepest descent loop to make a plot

Let's alter the preceding loop to accumulate selected epoch & loss values in lists for plotting.

+ Set `N_batch` and `n_epochs` to be larger, say, $500$ and $15,000$ respectively.
+ Use a smaller learning rate, say $\eta=0.00005$
+ Change the preceding `for` loop so that:
    + The `epoch` counter and the loss value are accumulated into lists every, say, `SKIP` iterations where `SKIP==500`.
    + Eliminate the `print` statement(s) to save on output.
+ After the `for` loop terminates, make a `semilogy` plot to verify that the loss function is actually decreasing with sucessive epochs.
    + Use the list `epochs` to accumulate the `epoch` every 500 epochs.
    + Use the list `losses` to accumulate the values of the loss function every 500 epochs.

In [ ]:
N_batch = 500
n_epochs = 15000
SKIP = 500
dimensions = [784, 15, 10]
N_samples = 60000
X = np.random.rand(dimensions[0], N_samples)
y = np.random.rand(dimensions[-1], N_samples)
eta = 0.00001
model = initialize_model(dimensions)
g = np.random.default_rng()
samples = np.arange(N_samples)

# accumulate the epochs, losses, & gradient norms in these respective lists
epochs, losses, norms = [], [], []
for epoch in range(n_epochs):
    pass
    # your code here ...

**Solution:**
```python
N_batch = 500
n_epochs = 15000
SKIP = 500
dimensions = [784, 15, 10]
N_samples = 60000
X = np.random.rand(dimensions[0], N_samples)
y = np.random.rand(dimensions[-1], N_samples)
eta = 0.00001
model = initialize_model(dimensions)
g = np.random.default_rng()
samples = np.arange(N_samples)

 # accumulate the epochs, losses, & gradient norms in these respective lists
epochs, losses, norms = [], [], []
for epoch in range(n_epochs):
    batch = sorted(g.choice(samples, size=N_batch, replace=False))
    yhat, model = forward(X[:,batch], model)
    model = backward(y[:,batch], model)
    if (divmod(epoch, SKIP)[1]==0):
        err = loss(yhat, y[:,batch])
        grad_norm = sum([np.linalg.norm(a.ravel())**2 for a in model['grad_weights']])
        grad_norm += sum([np.linalg.norm(a.ravel())**2 for a in model['grad_biases']])
        epochs.append(epoch)
        losses.append(err)
        norms.append(np.sqrt(grad_norm))
    model = update(eta, model)

 # code for plotting once that the lists epochs and losses are accumulated
fig, ax1 = plt.subplots()
ax1.set_xlim([0,n_epochs]); ax1.set_ylim([min(losses), max(losses)]);
ax1.set_xticks(epochs[::500]); ax1.set_xlabel("Epochs");
ax1.set_ylabel(r'$\mathcal{L}$');
h1 = ax1.plot(epochs, losses, 'r-', label=r'$\mathcal{L}$')
ax1.grid(True)
ax2 = ax1.twinx()
ax2.semilogy(epochs, norms, 'b', label='gradient')
ax2.set_ylabel('gradient')
ax2.grid(True)
fig.legend(loc=(0.65,0.75));
```

This is in fact [Stochastic Gradient Descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) for this particular loss function.

---

## Using our neural network on MNIST data

+  [MNIST database](https://en.wikipedia.org/wiki/MNIST_database) widely used as test problem in machine learning
 + hand-written decimal numerals ("digits")
 + $60,000$ training images, $10,000$ test images
 + normalized as $28\times28$ grayscale images

<center>
    <tr>
    <td><img src="./images/mnist.png" width="50%"></img></td>
    </tr>
</center>

+ Stored compressed data locally in `.npz` format

In [ ]:
DATA_PATH = pathlib.Path.cwd() / 'data' / 'mnist.npz'
d_file = np.load(DATA_PATH)

In [ ]:
for k in d_file.keys():
    arr = d_file[k]
    print(f'Array: {k}\tShape: {arr.shape} \tMin: {arr.min():3d}\tMax: {arr.max():3d}')

In [ ]:
X_train_orig = d_file['X_train']
X_test_orig = d_file['X_test']
y_train_orig = d_file['y_train']
y_test_orig = d_file['y_test']

In [ ]:
# Extract a single image
idx = 45621
digit_image = X_train_orig[idx]
plt.imshow(digit_image, cmap='gray')
plt.axis('off')
plt.title('Associated label: {}'.format(y_train_orig[idx]));

#### Exercise: Preprocessing the Digit Features

As a first step, preprocess the features in the arrays `X_train_orig` & `X_test_orig`.

+ Reshape the three-dimensional arrays into two-dimensional arrays.
 + Numpy arrays have a method for reshaping (or use the function `np.reshape`).
+ Rescale the integer values to be real values between 0 and 1.
 + Divide the arrays by 255.0 (the grayscale images have integer values between 0 & 255 by default).
+ Bind the rescaled & reshaped training & testing arrays to `X_train` & `X_test` respectively.

In [ ]:
# your code here...

**Solution:**
```python
N_train, nx, ny = X_train_orig.shape
X_train = X_train_orig.reshape((N_train, nx*ny))/255.0
N_test = len(X_test_orig)
X_test = X_test_orig.reshape((N_test, nx*ny))/255.0
```

In [ ]:
### For verification:
print('X_train: {}\t{}\t{}'.format(X_train.shape, X_train.min(), X_train.max()))
print('X_test:  {}\t{}\t{}'.format(X_test.shape, X_test.min(), X_test.max()))

#### Exercise: Preprocessing the Targets

As a second step, preprocess the targets `y_train_orig` & `y_test_orig` by converting them to two-dimensional arrays with one-hot encoded rows (corresponding to each categorical label).
+ The function [`OneHotEncoder` class from `sklearn.preprocessing`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) will do this for you.
+ Bind the results to `y_train` and `y_test`.
+ Warning: some reshaping to 2D arrays is required for compatibility.
+ Warning: the `OneHotEncoder` class produces sparse matrices by default; use the `sparse` keyword to provide a dense array instead.

In [ ]:
### Convert the target arrays y_train_orig & y_test_orig
### Assign the results to y_train & y_test respectively.
# your code here...

**Solution:**
```python
 ### Convert the target arrays y_train_orig & y_test_orig
 ### Assign the results to y_train & y_test respectively.
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False).fit(y_train_orig[:,np.newaxis])
y_train = encoder.transform(y_train_orig[:,np.newaxis])
y_test = encoder.transform(y_test_orig[:,np.newaxis])
```

In [ ]:
### For verification:
print('y_train: {}'.format(y_train.shape))
print('y_test:  {}'.format(y_test.shape))
print(y_test[:3]) # First three rows
print(y_test_orig[:3])

Now, assuming that the functions 
+ `initialize_model`;
+ `forward`;
+ `backward`; and
+ `update`

have been defined above, the steepest descent loop can be executed here:

In [ ]:
# Initialization
N_batch = 500
n_epochs = 15000
SKIP = 500
dimensions = [784, 15, 10]
N_samples = 60000
X = X_train.T  # Transpose required to conform in this case
y = y_train.T  # Transpose required to conform in this case
eta = 0.00005
model = initialize_model(dimensions)
g = np.random.default_rng()
samples = np.arange(N_samples)

In [ ]:
# accumulate the epochs, losses, & gradient norms in these respective lists
epochs, losses, norms = [], [], []
for epoch in range(n_epochs):
    batch = sorted(g.choice(samples, size=N_batch, replace=False))
    yhat, model = forward(X[:,batch], model)
    model = backward(y[:,batch], model)
    if (divmod(epoch, SKIP)[1]==0):
        err = loss(yhat, y[:,batch])
        grad_norm = sum([np.linalg.norm(a.ravel())**2 for a in model['grad_weights']])
        grad_norm += sum([np.linalg.norm(a.ravel())**2 for a in model['grad_biases']])
        epochs.append(epoch)
        losses.append(err)
        norms.append(np.sqrt(grad_norm))
    model = update(eta, model)

# code for plotting once that the lists epochs and losses are accumulated
fig, ax1 = plt.subplots()
ax1.set_xlim([0,n_epochs]); ax1.set_ylim([min(losses), max(losses)]);
ax1.set_xticks(epochs[::500]); ax1.set_xlabel("Epochs");
ax1.set_ylabel(r'$\mathcal{L}$');
h1 = ax1.plot(epochs, losses, 'r-', label=r'$\mathcal{L}$')
ax1.grid(True)
ax2 = ax1.twinx()
ax2.semilogy(epochs, norms, 'b', label='gradient')
ax2.set_ylabel('gradient')
ax2.grid(True)
fig.legend(loc=(0.65,0.75));    

## Summary

- Essential components of neural networks: activation functions, layers, forward & backward propagation
- Building a neural network in NumPy
  - Hand-coding stochastic gradient descent

---

<center>
    <tr>
    <td><img src="images/Quansight_Logo_Lockup_1.png" width="25%"></img></td>
    </tr>
</center>